In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Activation
from keras.layers import Dropout, RepeatVector, TimeDistributed
from keras import Input, Model


In [ ]:
'tensorflow==2.7.0',
'tf-models-official==2.7.0',
'tensorflow_io==0.23.1'

In [ ]:
#Train and test sets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 

datagen= ImageDataGenerator(
    rescale = 1./255,
    #horizontal_flip =True,
    #vertical_flip = True,
    validation_split = 0.2 
)
#Name of the folder that contains the images for the training
name_directory = ''

data_gen_train =datagen.flow_from_directory(name_directory, target_size=(224,224), batch_size=32, shuffle=True, subset='training')
data_gen_test = datagen.flow_from_directory(name_directory, target_size=(224, 224), batch_size=32, shuffle=True, subset='validation')

In [ ]:
base_model=tf.keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

In [ ]:
base_model.trainable=False

In [ ]:
#Fine-tuning
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(320, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(320, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

seq_length = 15
input_dims = 320
output_dims = 2
n_hidden = 320
lstm_dims = 1000
model1_inputs = Input(shape=(input_dims,))
model1_outputs = Input(shape=(output_dims,))

net1 = Dense(n_hidden)(model1_inputs)
net1 = Dense(n_hidden)(net1)

net1 = RepeatVector(3)(net1)
net1 = LSTM(lstm_dims, return_sequences=True)(net1)
net1 = LSTM(lstm_dims, return_sequences=False)(net1)
net1 = Dense(output_dims, activation='softmax')(net1)
model1_outputs = net1

model1 = Model(inputs=model1_inputs, outputs = model1_outputs, name='model1')

model1.summary()

model.add(model1)

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
#Train
Epochs = 200

history = model.fit(data_gen_train, epochs=Epochs, batch_size=32, validation_data=data_gen_test)

In [ ]:
import matplotlib.pyplot as plt
acc = historial.history['accuracy']
val_acc = historial.history['val_accuracy']

loss = historial.history['loss']
val_loss = historial.history['val_loss']

range_epochs = range(Epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range_epochs, acc, label='Accuracy train')
plt.plot(range_epochs, val_acc, label='Acurracy test')
plt.legend(loc='lower right')
plt.title('Accuracy test and train')

plt.subplot(1,2,2)
plt.plot(range_epochs, loss, label='Loss Train')
plt.plot(range_epochs, val_loss, label='Loss test')
plt.legend(loc='upper right')
plt.title('Loss test and train')
plt.show()

In [ ]:
#Saving the model
dir='./model_mobilenetv2/'

if not os.path.exists(dir):
    os.mkdir(dir)
model.save('./model_mobilenetv2/model_mobilenetv2_LSTM.h5')
model.save_weights('./model_mobilenetv2/weights_mobilenetv2_LSTM.h5')